In [1]:
#1
!nvidia-smi

Sat Aug  9 22:27:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
#2
# Install core libraries

!pip install -q -U "transformers==4.47.1" "datasets==3.2.0" "accelerate==1.2.1" \
               "trl==0.14.0" "peft==0.14.0" "huggingface_hub>=0.25.0" "math_verify==0.3.3"

In [3]:
#3
# Login to Hugging Face 
from huggingface_hub import notebook_login

notebook_login()  

In [4]:
#4
# Load the dataset
from datasets import load_dataset

dataset_id = "mohammad-shirkhani/social_movielens_compress"

train_full = load_dataset(dataset_id, split="train")

start = 500
count = 500
end = min(len(train_full), start + count)
indices = list(range(start, end))

train_dataset = train_full.select(indices)

print(f"Loaded {dataset_id} — train slice [{start}:{end}) with {len(train_dataset)} rows.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/33.3M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/4.15M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/4.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/80000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Loaded mohammad-shirkhani/social_movielens_compress — train slice [500:1000) with 500 rows.


In [5]:
#5
# Print dataset structure
print(train_dataset)

Dataset({
    features: ['user', 'item', 'answer', 'paths'],
    num_rows: 500
})


In [6]:
#6
# Print the first example to see raw fields
print(train_dataset[0])

{'user': {'UserID': 72, 'Age': 48, 'Gender': 'Female', 'Occupation': 'administrator'}, 'item': {'MovieID': 2, 'Title': 'GoldenEye (1995)', 'ReleaseDate': '01-Jan-1995', 'Genres': 'Action, Adventure, Thriller'}, 'answer': 3.0, 'paths': ['user_question -> user_item_4 (rating=4) -> Item{MovieID 196, Title "Dead Poets Society (1989)", Release Date 01-Jan-1989, Genres Drama} -> item_user_5 (rating=5) -> User{UserID 87, Age 47, Gender Male, Occupation administrator} -> user_item_4 (rating=4) -> item_question', 'user_question -> user_item_5 (rating=5) -> Item{MovieID 427, Title "To Kill a Mockingbird (1962)", Release Date 01-Jan-1962, Genres Drama} -> item_user_4 (rating=4) -> User{UserID 42, Age 30, Gender Male, Occupation administrator} -> user_item_4 (rating=4) -> Item{MovieID 684, Title "In the Line of Fire (1993)", Release Date 01-Jan-1993, Genres Action, Thriller} -> itemsim -> item_question', 'user_question -> user_item_3 (rating=3) -> Item{MovieID 1110, Title "Tank Girl (1995)", Relea

In [7]:
#7
import json

# Helper
def to_json_str(d):
    return json.dumps(d, ensure_ascii=False, sort_keys=False)

def build_problem(example):
    """
    Build an English prompt that:
    - Defines the task (predict rating with 0.1 precision in [1.0, 5.0]).
    - Shows user and item features.
    - Explains the bipartite graph and meta-paths.
    - Lists the meta-paths line-by-line for evidence-based prediction.
    """
    user_block = to_json_str(example.get("user", {}))
    item_block = to_json_str(example.get("item", {}))

    # Ensure paths are a list of strings
    paths_list = example.get("paths", []) or []
    # Normalize and make each path on its own line with a leading dash
    if isinstance(paths_list, (list, tuple)):
        cleaned = [str(p).strip() for p in paths_list if p is not None]
        paths_str = "\n".join(f"- {p}" for p in cleaned)
    else:
        # Fallback if it's a single string (unlikely)
        paths_str = f"- {str(paths_list).strip()}"

    prompt = (
        "You are a model that predicts the rating a user will give to a movie. "
        "Your task is to predict this user's rating for this movie in the range [1.0, 5.0] "
        "with a precision of one decimal place (e.g., 3.2 or 4.7). "
        "The result must be logical, evidence-based, and derived from the provided meta paths.\n\n"
        "Constraints and output format:\n"
        "• The rating must be between 1.0 and 5.0 (inclusive).\n"
        "• Output the rating with exactly one decimal place.\n"
        "• At the end, answer only one number (with one decimal place) as the 'Predicted Rating'.\n\n"
        "User features:\n"
        f"{user_block}\n\n"
        "Movie features:\n"
        f"{item_block}\n\n"
        "Reasoning guide:\n"
        "• The graph is bipartite (users–movies). Users similar to each other are connected with the 'usersim' edge, "
        "and similar movies are connected with the 'itemsim' edge.\n"
        "• Edges 'user_item_k' and 'item_user_k' indicate past ratings (rating=k) along the paths.\n"
        "• Use the paths below as positive/negative evidence; consider user similarity, movie similarity, genres, "
        "and rating patterns.\n"
        "• If evidence is contradictory, weigh the frequency and strength of the signals "
        "(path length, proximity, genre match, and rating patterns) to reach a conclusion.\n"
        "• After reasoning, round the final number to one decimal place within the allowed range.\n"
        "answer: Only a single number with one decimal place as the predicted rating.\n\n"
        "Meta Paths:\n"
        f"{paths_str}\n\n"
    )
    return {"problem": prompt}

# Add the new column
train_dataset = train_dataset.map(build_problem, desc="Building 'problem' prompts")


Building 'problem' prompts:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
#8
from pprint import pprint

pprint(train_dataset[0])

{'answer': 3.0,
 'item': {'Genres': 'Action, Adventure, Thriller',
          'MovieID': 2,
          'ReleaseDate': '01-Jan-1995',
          'Title': 'GoldenEye (1995)'},
 'paths': ['user_question -> user_item_4 (rating=4) -> Item{MovieID 196, Title '
           '"Dead Poets Society (1989)", Release Date 01-Jan-1989, Genres '
           'Drama} -> item_user_5 (rating=5) -> User{UserID 87, Age 47, Gender '
           'Male, Occupation administrator} -> user_item_4 (rating=4) -> '
           'item_question',
           'user_question -> user_item_5 (rating=5) -> Item{MovieID 427, Title '
           '"To Kill a Mockingbird (1962)", Release Date 01-Jan-1962, Genres '
           'Drama} -> item_user_4 (rating=4) -> User{UserID 42, Age 30, Gender '
           'Male, Occupation administrator} -> user_item_4 (rating=4) -> '
           'Item{MovieID 684, Title "In the Line of Fire (1993)", Release Date '
           '01-Jan-1993, Genres Action, Thriller} -> itemsim -> item_question',
           

In [9]:
#9

SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

def make_conversation(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
    }

# Map only on the training dataset
train_dataset = train_dataset.map(make_conversation, desc="Building conversation prompts for train")


Building conversation prompts for train:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
#10
print(train_dataset[0]["prompt"])

[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>', 'role': 'system'}, {'content': 'You are a model that predicts the rating a user will give to a movie. Your task is to predict this user\'s rating for this movie in the range [1.0, 5.0] with a precision of one decimal place (e.g., 3.2 or 4.7). The result must be logical, evidence-based, and derived from the provided meta paths.\n\nConstraints and output format:\n• The rating must be between 1.0 and 5.0 (inclusive).\n• Output the rating with exactly one decimal place.\n• At the end, answer only one number (with one decimal place) as the \'Predicted Rating\'.\n\nUser features:\n{"Us

In [11]:
#11
# Verify the original problem we constructed earlier
print(train_dataset[0]["problem"])

You are a model that predicts the rating a user will give to a movie. Your task is to predict this user's rating for this movie in the range [1.0, 5.0] with a precision of one decimal place (e.g., 3.2 or 4.7). The result must be logical, evidence-based, and derived from the provided meta paths.

Constraints and output format:
• The rating must be between 1.0 and 5.0 (inclusive).
• Output the rating with exactly one decimal place.
• At the end, answer only one number (with one decimal place) as the 'Predicted Rating'.

User features:
{"UserID": 72, "Age": 48, "Gender": "Female", "Occupation": "administrator"}

Movie features:
{"MovieID": 2, "Title": "GoldenEye (1995)", "ReleaseDate": "01-Jan-1995", "Genres": "Action, Adventure, Thriller"}

Reasoning guide:
• The graph is bipartite (users–movies). Users similar to each other are connected with the 'usersim' edge, and similar movies are connected with the 'itemsim' edge.
• Edges 'user_item_k' and 'item_user_k' indicate past ratings (ratin

In [12]:
#12
print(train_dataset)

Dataset({
    features: ['user', 'item', 'answer', 'paths', 'problem', 'prompt'],
    num_rows: 500
})


In [13]:
#13
train_dataset = train_dataset.remove_columns(["user", "problem" , "item", "paths"])
print(train_dataset)

Dataset({
    features: ['answer', 'prompt'],
    num_rows: 500
})


In [14]:
#14
# Load the previously trained LoRA adapter
import torch
from peft import AutoPeftModelForCausalLM

adapter_repo_id = "mohammad-shirkhani/Qwen2.5-1.5B-GRPO-rating"  # previous stage repo

model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_repo_id,
    torch_dtype=torch.bfloat16,   
    device_map="auto",
    is_trainable=True,            
)

print(f"Loaded PEFT adapter from: {adapter_repo_id}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

adapter_config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/69.8M [00:00<?, ?B/s]

Loaded PEFT adapter from: mohammad-shirkhani/Qwen2.5-1.5B-GRPO-rating


In [15]:
#15
# No new LoRA config here
model.print_trainable_parameters()

try:
    print("Active adapters:", getattr(model, "active_adapters", None))
except Exception:
    pass


trainable params: 17,432,576 || all params: 1,560,730,624 || trainable%: 1.1169
Active adapters: ['default']


In [16]:
#16
# Format reward that enforces the presence of <think>...</think> followed by <answer>...</answer>.

import re
from typing import Any, List

_FORMAT_RE = re.compile(r"^<think>[\s\S]*?</think>\s*<answer>[\s\S]*?</answer>\s*$")

def _first_completion_text(completion: Any) -> str:
    """
    Try to extract the assistant content from a TRL completion.
    Handles a few shapes:
      - str
      - dict with 'content'
      - list[dict] chat messages -> prefer the last assistant message if present; otherwise the first content-like
      - anything else -> fallback to str()
    """
    if isinstance(completion, str):
        return completion
    if isinstance(completion, dict):
        return str(completion.get("content", ""))
    if isinstance(completion, (list, tuple)):
        for msg in reversed(completion):
            if isinstance(msg, dict) and msg.get("role") == "assistant" and "content" in msg:
                return str(msg["content"])
        for msg in completion:
            if isinstance(msg, dict) and "content" in msg:
                return str(msg["content"])
        return str(completion[0]) if completion else ""
    return str(completion)

def format_reward(completions: List[Any], **kwargs) -> List[float]:
    """
    Reward function that checks if the completion follows:
      <think> ... </think> <answer> ... </answer>
    Returns 1.0 if the format is correct, 0.0 otherwise.
    """
    rewards = []
    for completion in completions:
        text = _first_completion_text(completion)
        text = text.strip()
        rewards.append(1.0 if _FORMAT_RE.match(text) else 0.0)
    return rewards


In [17]:
#17
# Accuracy reward 

import re
from typing import Any, List

_ANS_NUM_RE = re.compile(r"<answer>\s*([+-]?\d+(?:\.\d+)?)\s*</answer>", flags=re.IGNORECASE)

def _extract_answer_float(text: str):
    """
    Extract the first numeric value inside <answer>...</answer>.
    Returns float(value) or None if not found / not parseable.
    """
    m = _ANS_NUM_RE.search(text)
    if not m:
        return None
    try:
        return float(m.group(1))
    except Exception:
        return None

def accuracy_reward(completions: List[Any], **kwargs) -> List[float]:
    """
    Reward = -|y_true - y_pred|
    If no numeric prediction is present inside <answer>...</answer>, reward = -5.0.
    """
    gts = kwargs.get("answer", [])
    rewards: List[float] = []

    for idx, completion in enumerate(completions):
        text = _first_completion_text(completion).strip()
        y_true_raw = gts[idx] if idx < len(gts) else None

        # Convert ground truth to float if possible
        try:
            y_true = float(y_true_raw)
        except Exception:
            rewards.append(0.0)
            continue

        y_pred = _extract_answer_float(text)
        if y_pred is None:
            # No numeric answer in <answer>...</answer>
            rewards.append(-5.0)
        else:
            # Negative absolute error 
            rewards.append(-abs(y_true - y_pred))

    return rewards


In [18]:
#18
from trl import GRPOConfig

training_args = GRPOConfig(
    output_dir="Qwen2.5-1.5B-GRPO-rating-1000",
    learning_rate=1e-5,                  
    remove_unused_columns=False,      
    per_device_train_batch_size=1,     
    gradient_accumulation_steps=8,      
    num_train_epochs=1,                  
    bf16=True,                           
    max_completion_length=1024,         
    num_generations=4,                  
    max_prompt_length=1024,            
    # Logging / saving
    report_to=["tensorboard"],
    logging_steps=10,
    push_to_hub=True,                  
    save_strategy="steps",
    save_steps=200,                     
)
print(training_args)

GRPOConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
beta=0.04,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval

In [21]:
#19
# Create the GRPO trainer 

from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model, reward_funcs=[format_reward, accuracy_reward], args=training_args, train_dataset=train_dataset
)

print("GRPOTrainer is ready.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

GRPOTrainer is ready.


In [22]:
#20
# Start training

trainer.train()

Step,Training Loss
10,0.001500
20,0.001900
30,0.002000
40,0.003900
50,0.003200
60,0.002800


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=62, training_loss=0.002517987391911447, metrics={'train_runtime': 1073.7709, 'train_samples_per_second': 0.466, 'train_steps_per_second': 0.058, 'total_flos': 0.0, 'train_loss': 0.002517987391911447})

In [23]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(commit_message=f"Upload LoRA adapter + tokenizer for {base_model_id}")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...-GRPO-rating-1000/training_args.bin: 100%|##########| 5.56kB / 5.56kB            

  ...ents.1754778655.7290ebda73a0.2210.0: 100%|##########| 9.39kB / 9.39kB            

...

  ...ents.1754778655.7290ebda73a0.2210.0: 100%|##########| 9.39kB / 9.39kB            

  ....5B-GRPO-rating-1000/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...ting-1000/adapter_model.safetensors:   4%|4         | 41.9MB / 1.00GB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mohammad-shirkhani/Qwen2.5-1.5B-GRPO-rating-1000/commit/a90e15905ed7aa4327d9a787385814a8a866d56d', commit_message='Upload LoRA adapter + tokenizer for Qwen/Qwen2.5-1.5B-Instruct', commit_description='', oid='a90e15905ed7aa4327d9a787385814a8a866d56d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mohammad-shirkhani/Qwen2.5-1.5B-GRPO-rating-1000', endpoint='https://huggingface.co', repo_type='model', repo_id='mohammad-shirkhani/Qwen2.5-1.5B-GRPO-rating-1000'), pr_revision=None, pr_num=None)